# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 20 2022 8:30AM,242267,10,0085889538,ISDIN Corporation,Released
1,Jul 20 2022 8:30AM,242267,10,0085889537,ISDIN Corporation,Released
2,Jul 20 2022 8:30AM,242265,10,0085889589,ISDIN Corporation,Released
3,Jul 20 2022 8:30AM,242267,10,0085889590,ISDIN Corporation,Released
4,Jul 20 2022 8:30AM,242265,10,0085889601,ISDIN Corporation,Released
5,Jul 20 2022 8:30AM,242267,10,0085889606,ISDIN Corporation,Released
6,Jul 20 2022 8:30AM,242265,10,0085889603,ISDIN Corporation,Released
7,Jul 20 2022 8:30AM,242267,10,0085889637,ISDIN Corporation,Released
8,Jul 20 2022 8:30AM,242267,10,0085889641,ISDIN Corporation,Released
9,Jul 20 2022 8:30AM,242267,10,0085889660,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
19,242265,Released,10
20,242267,Released,13
21,242268,Released,1
22,242269,Released,1
23,242270,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
242265,NaN,NaN,10.0
242267,NaN,NaN,13.0
242268,NaN,NaN,1.0
242269,NaN,NaN,1.0
242270,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242201,0.0,0.0,14.0
242207,0.0,0.0,1.0
242208,0.0,0.0,1.0
242209,0.0,1.0,0.0
242214,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242201,0,0,14
242207,0,0,1
242208,0,0,1
242209,0,1,0
242214,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242201,0,0,14
1,242207,0,0,1
2,242208,0,0,1
3,242209,0,1,0
4,242214,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242201,,,14
1,242207,,,1
2,242208,,,1
3,242209,,1,
4,242214,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 20 2022 8:30AM,242267,10,ISDIN Corporation
2,Jul 20 2022 8:30AM,242265,10,ISDIN Corporation
22,Jul 20 2022 8:30AM,242263,10,"Snap Medical Industries, LLC"
35,Jul 20 2022 8:30AM,242270,10,"Citieffe, Inc."
36,Jul 19 2022 4:33PM,242269,18,Innova Softgel LLC
37,Jul 19 2022 4:18PM,242268,16,"SHL Pharma, LLC"
38,Jul 19 2022 3:39PM,242262,19,ACG North America LLC
39,Jul 19 2022 3:28PM,242259,16,Sartorius Bioprocess Solutions
41,Jul 19 2022 3:28PM,242258,10,Emerginnova
43,Jul 19 2022 3:16PM,242255,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 20 2022 8:30AM,242267,10,ISDIN Corporation,,,13
1,Jul 20 2022 8:30AM,242265,10,ISDIN Corporation,,,10
2,Jul 20 2022 8:30AM,242263,10,"Snap Medical Industries, LLC",,,12
3,Jul 20 2022 8:30AM,242270,10,"Citieffe, Inc.",,,1
4,Jul 19 2022 4:33PM,242269,18,Innova Softgel LLC,,,1
5,Jul 19 2022 4:18PM,242268,16,"SHL Pharma, LLC",,,1
6,Jul 19 2022 3:39PM,242262,19,ACG North America LLC,,,1
7,Jul 19 2022 3:28PM,242259,16,Sartorius Bioprocess Solutions,,,2
8,Jul 19 2022 3:28PM,242258,10,Emerginnova,,,2
9,Jul 19 2022 3:16PM,242255,10,ISDIN Corporation,,,12


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 20 2022 8:30AM,242267,10,ISDIN Corporation,13,,
1,Jul 20 2022 8:30AM,242265,10,ISDIN Corporation,10,,
2,Jul 20 2022 8:30AM,242263,10,"Snap Medical Industries, LLC",12,,
3,Jul 20 2022 8:30AM,242270,10,"Citieffe, Inc.",1,,
4,Jul 19 2022 4:33PM,242269,18,Innova Softgel LLC,1,,
5,Jul 19 2022 4:18PM,242268,16,"SHL Pharma, LLC",1,,
6,Jul 19 2022 3:39PM,242262,19,ACG North America LLC,1,,
7,Jul 19 2022 3:28PM,242259,16,Sartorius Bioprocess Solutions,2,,
8,Jul 19 2022 3:28PM,242258,10,Emerginnova,2,,
9,Jul 19 2022 3:16PM,242255,10,ISDIN Corporation,12,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 20 2022 8:30AM,242267,10,ISDIN Corporation,13,,
1,Jul 20 2022 8:30AM,242265,10,ISDIN Corporation,10,,
2,Jul 20 2022 8:30AM,242263,10,"Snap Medical Industries, LLC",12,,
3,Jul 20 2022 8:30AM,242270,10,"Citieffe, Inc.",1,,
4,Jul 19 2022 4:33PM,242269,18,Innova Softgel LLC,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 20 2022 8:30AM,242267,10,ISDIN Corporation,13.0,NaN,NaN
1,Jul 20 2022 8:30AM,242265,10,ISDIN Corporation,10.0,NaN,NaN
2,Jul 20 2022 8:30AM,242263,10,"Snap Medical Industries, LLC",12.0,NaN,NaN
3,Jul 20 2022 8:30AM,242270,10,"Citieffe, Inc.",1.0,NaN,NaN
4,Jul 19 2022 4:33PM,242269,18,Innova Softgel LLC,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 20 2022 8:30AM,242267,10,ISDIN Corporation,13.0,0.0,0.0
1,Jul 20 2022 8:30AM,242265,10,ISDIN Corporation,10.0,0.0,0.0
2,Jul 20 2022 8:30AM,242263,10,"Snap Medical Industries, LLC",12.0,0.0,0.0
3,Jul 20 2022 8:30AM,242270,10,"Citieffe, Inc.",1.0,0.0,0.0
4,Jul 19 2022 4:33PM,242269,18,Innova Softgel LLC,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1695785,51.0,0.0,0.0
16,484527,3.0,0.0,0.0
18,484493,2.0,0.0,0.0
19,1211221,20.0,9.0,6.0
20,726643,33.0,1.0,0.0
22,484422,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1695785,51.0,0.0,0.0
1,16,484527,3.0,0.0,0.0
2,18,484493,2.0,0.0,0.0
3,19,1211221,20.0,9.0,6.0
4,20,726643,33.0,1.0,0.0
5,22,484422,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,51.0,0.0,0.0
1,16,3.0,0.0,0.0
2,18,2.0,0.0,0.0
3,19,20.0,9.0,6.0
4,20,33.0,1.0,0.0
5,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,51.0
1,16,Released,3.0
2,18,Released,2.0
3,19,Released,20.0
4,20,Released,33.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,18,19,20,22
Status,,,,,,
Completed,0.0,0.0,0.0,6.0,0.0,0.0
Executing,0.0,0.0,0.0,9.0,1.0,0.0
Released,51.0,3.0,2.0,20.0,33.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,18,19,20,22
0,Completed,0.0,0.0,0.0,6.0,0.0,0.0
1,Executing,0.0,0.0,0.0,9.0,1.0,0.0
2,Released,51.0,3.0,2.0,20.0,33.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,18,19,20,22
0,Completed,0.0,0.0,0.0,6.0,0.0,0.0
1,Executing,0.0,0.0,0.0,9.0,1.0,0.0
2,Released,51.0,3.0,2.0,20.0,33.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()